# Strands エージェントと AgentCore Memory（短期メモリ）- MemoryManager 使用


## はじめに

このチュートリアルでは、Strands エージェントと AgentCore の**短期メモリ**を **MemoryManager** および **MemorySessionManager** を使用して**パーソナルエージェント**を構築する方法を紹介します。エージェントは `get_last_k_turns` を使用してセッション内の最近の会話を記憶し、ユーザーが戻ってきた際にシームレスに会話を継続できます。

**注意: これは MemoryManager と MemorySessionManager を使用した短期メモリサンプルバージョンです。**


### チュートリアルの詳細

| 情報               | 詳細                                                                            |
|:-------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | 短期会話メモリ                                                                  |
| エージェントタイプ   | パーソナルエージェント                                                          |
| エージェントフレームワーク | Strands Agents                                                                 |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                     |
| チュートリアルコンポーネント | MemoryManager 付き AgentCore 短期メモリ、AgentInitializedEvent と MessageAddedEvent フック |
| 難易度              | 初級                                                                            |

学習内容：
- MemoryManager を使用した会話継続性のための短期メモリの使用
- MemorySessionManager を使用した直近 K ターンの会話履歴の取得
- リアルタイム情報取得のための Web 検索ツール
- セッション管理を使用した会話履歴付きエージェントの初期化
- MemoryClient から MemoryManager アーキテクチャへの移行支援

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件

このチュートリアルを実行するには：
- Python 3.10以上
- AgentCore Memory 権限を持つ AWS 認証情報
- MemoryManager サポート付き Amazon Bedrock AgentCore SDK
- Amazon Bedrock モデルへのアクセス

環境のセットアップから始めましょう！

## ステップ 1: セットアップとインポート

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime
from botocore.exceptions import ClientError

# ロギングの設定
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("personal-agent")

In [ ]:
# Strands Agent に必要なモジュールをインポート
import os
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent

# メモリ管理モジュールをインポート
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory.constants import ConversationalMessage, MessageRole
from bedrock_agentcore.memory.session import MemorySession, MemorySessionManager

# メッセージロール定数を定義
USER = MessageRole.USER
ASSISTANT = MessageRole.ASSISTANT

# 設定
REGION = os.getenv('AWS_REGION', 'us-east-1') # エージェントの AWS リージョン
ACTOR_ID = "user_123" # 任意の一意識別子（AgentID、User ID など）
SESSION_ID = "personal_session_001" # 一意のセッション識別子

# IAM ロール作成用に boto3 をインポート
import boto3
import json as json_module

## ステップ 2: Web 検索ツール

まず、エージェント用のシンプルな Web 検索ツールを作成しましょう。元の実装と変わりません。

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """最新情報を取得するために Web を検索します。
    
    Args:
        keywords (str): 検索クエリのキーワード。
        region (str): 検索リージョン: wt-wt, us-en, uk-en, ru-ru など。
        max_results (int | None): 返す結果の最大数。
    Returns:
        検索結果の辞書リスト。
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

## ステップ 3: MemoryManager を使用した Memory リソースの作成

短期メモリの場合、MemoryManager を使用してストラテジーなしでメモリリソースを作成します。これにより、`get_last_k_turns` で取得できる生の会話ターンが保存されます。

**注意: このセクションでは、従来の MemoryClient の代わりに MemoryManager アーキテクチャを使用しています。**

In [ ]:
# Memory Manager を初期化
memory_manager = MemoryManager(region_name=REGION)
memory_name = "PersonalAgentMemoryManager"

logger.info(f"✅ MemoryManager initialized for region: {REGION}")
logger.info(f"Memory manager type: {type(memory_manager)}")

# MemoryManager を使用してメモリリソースを作成
logger.info(f"Creating memory '{memory_name}' for short-term conversational storage...")

try:
    memory = memory_manager.get_or_create_memory(
        name=memory_name,
        strategies=[],  # 短期メモリ用のストラテジーなし
        description="Short-term memory for personal agent",
        event_expiry_days=7,  # 短期メモリの保持期間
        memory_execution_role_arn=None,  # 短期メモリではオプション
    )
    memory_id = memory.id
    logger.info(f"✅ Successfully created/retrieved memory with MemoryManager:")
    logger.info(f"   Memory ID: {memory_id}")
    logger.info(f"   Memory Name: {memory.name}")
    logger.info(f"   Memory Status: {memory.status}")
    
except Exception as e:
    # 拡張エラーレポートでメモリ作成中のエラーを処理
    logger.error(f"❌ Memory creation failed: {e}")
    logger.error(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    
    # エラー時のクリーンアップ - 部分的に作成されたメモリを削除
    if 'memory_id' in locals():
        try:
            logger.info(f"Attempting cleanup of partially created memory: {memory_id}")
            memory_manager.delete_memory(memory_id)
            logger.info(f"✅ Successfully cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"❌ Failed to clean up memory: {cleanup_error}")
    
    # 元の例外を再スロー
    raise

## ステップ 4: Session Manager の初期化

このセクションでは、セッションベースのメモリ操作のための MemorySessionManager を導入し、actor と session を管理する MemorySession を作成します。

In [ ]:
# セッションメモリマネージャーを初期化
session_manager = MemorySessionManager(memory_id=memory.id, region_name=REGION)

# 特定の actor/session の組み合わせ用にメモリセッションを作成
user_session = session_manager.create_memory_session(
    actor_id=ACTOR_ID, 
    session_id=SESSION_ID
)

logger.info(f"✅ Session manager initialized for memory: {memory.id}")
logger.info(f"✅ Memory session created for actor: {ACTOR_ID}, session: {SESSION_ID}")
logger.info(f"Session manager type: {type(session_manager)}")
logger.info(f"Memory session type: {type(user_session)}")

## ステップ 5: Memory Hook Provider

このステップでは、MemorySession を使用してメモリ操作を自動化するカスタム `MemoryHookProvider` クラスを定義します。フックは、エージェントの実行ライフサイクルの特定のポイントで実行される特別な関数です。作成するメモリフックには2つの主要な機能があります：
1. **最近の会話をロード**: エージェントが初期化されたときに自動的に最近の会話履歴をロードするための `AgentInitializedEvent` フック。
2. **最新のメッセージを保存**: セッションマネージャーを使用して新しい会話メッセージを保存。

**MemoryClient バージョンからの主な変更点：**
- MemoryClient の代わりに MemorySession を使用
- タプルの代わりに ConversationalMessage オブジェクトを使用
- create_event() の代わりに add_turns() を使用
- 型安全性のために MessageRole enum を使用

In [ ]:
class MemoryHookProvider(HookProvider):
    def __init__(self, memory_session: MemorySession):  # MemorySession を受け入れる
        self.memory_session = memory_session
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """MemorySession を使用してエージェント開始時に最近の会話履歴をロード"""
        try:
            # 事前設定済みのメモリセッションを使用（actor_id/session_id は不要）
            recent_turns = self.memory_session.get_last_k_turns(k=5)
            
            if recent_turns:
                # コンテキスト用に会話履歴をフォーマット
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        # EventMessage オブジェクトと dict 形式の両方を処理
                        if hasattr(message, 'role') and hasattr(message, 'content'):
                            role = message['role']
                            content = message['content']
                        else:
                            role = message.get('role', 'unknown')
                            content = message.get('content', {}).get('text', '')
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # エージェントのシステムプロンプトにコンテキストを追加
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                logger.info(f"✅ Loaded {len(recent_turns)} conversation turns using MemorySession")
                
        except Exception as e:
            logger.error(f"Memory load error: {e}")

    def on_message_added(self, event: MessageAddedEvent):
        """MemorySession を使用してメッセージをメモリに保存"""
        messages = event.agent.messages
        try:
            if messages and len(messages) > 0 and messages[-1]["content"][0].get("text"):
                message_text = messages[-1]["content"][0]["text"]
                message_role = MessageRole.USER if messages[-1]["role"] == "user" else MessageRole.ASSISTANT
                
                # メモリセッションインスタンスを使用（actor_id/session_id は渡す必要なし）
                result = self.memory_session.add_turns(
                    messages=[ConversationalMessage(message_text, message_role)]
                )
                
                event_id = result['eventId']
                logger.info(f"✅ Stored message with Event ID: {event_id}, Role: {message_role.value}")
                
        except Exception as e:
            logger.error(f"Memory save error: {e}")
            import traceback
            logger.error(f"Full traceback: {traceback.format_exc()}")
    
    def register_hooks(self, registry: HookRegistry):
        # メモリフックを登録
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)
        logger.info("✅ Memory hooks registered with MemorySession")


## ステップ 6: Web 検索機能付きパーソナルエージェントの作成

このエージェントは、MemorySessionManager から作成された MemorySession と連携する MemoryHookProvider を使用します。

In [ ]:
def create_personal_agent():
    """MemorySession を使用してメモリと Web 検索機能を持つパーソナルエージェントを作成"""
    agent = Agent(
        name="PersonalAssistant",
        model="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # または希望のモデル
        system_prompt=f"""あなたは Web 検索機能を持つ親切なパーソナルアシスタントです。
        
        あなたができること：
        - 一般的な質問と情報検索
        - 最新情報の Web 検索
        - 個人的なタスク管理
        
        最新情報が必要な場合は、websearch 関数を使用してください。
        今日の日付: {datetime.today().strftime('%Y-%m-%d')}
        フレンドリーかつプロフェッショナルに対応してください。""",
        hooks=[MemoryHookProvider(user_session)], 
        tools=[websearch],
    )
    return agent

# エージェントを作成
agent = create_personal_agent()
logger.info("✅ Personal agent created with MemorySession and web search")

#### おめでとうございます！MemoryManager と MemorySession を使用したエージェントの準備ができました！
## エージェントをテストしましょう

In [ ]:
# メモリ付き会話テスト
print("=== 最初の会話 ===")
print(f"ユーザー: 私の名前はアレックスで、AI について学ぶことに興味があります。")
print(f"エージェント: ", end="")
agent("My name is Alex and I'm interested in learning about AI.")

In [ ]:
print(f"ユーザー: 2025年の最新AIトレンドを検索してもらえますか？")
print(f"エージェント: ", end="")
agent("Can you search for the latest AI trends in 2025?")

In [ ]:
print(f"ユーザー: 特に機械学習の応用に興味があります。")
print(f"エージェント: ", end="")
agent("I'm particularly interested in machine learning applications.")

## MemorySessionManager を使用したメモリ継続性のテスト

メモリシステムが正しく動作しているかテストするために、エージェントの新しいインスタンスを作成し、MemorySessionManager を使用して以前に保存された情報にアクセスできるか確認します：

In [ ]:
# 新しいエージェントインスタンスを作成（ユーザーが戻ってきた状況をシミュレート）
print("=== ユーザーが戻ってきました - 新しいセッション ===")
new_agent = create_personal_agent()

# メモリの継続性をテスト
print(f"ユーザー: 私の名前は何でしたっけ？")
print(f"エージェント: ", end="")
new_agent("What was my name again?")

print(f"ユーザー: 機械学習についてもっと情報を検索してもらえますか？")
print(f"エージェント: ", end="")
new_agent("Can you search for more information about machine learning?")

## MemorySession を使用した保存メモリの確認

In [ ]:
# MemorySession を使用してメモリに保存されている内容を確認
print("=== Memory Contents ===")
recent_turns = user_session.get_last_k_turns(k=3) 

for i, turn in enumerate(recent_turns, 1):
    print(f"Turn {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
        print(f"  {role}: {content}")
    print()

## まとめ

このチュートリアルでは、MemorySessionManager と MemorySession の両方を使用したパーソナルエージェントの構築方法を紹介しました。学習した内容：

- **MemorySessionManager**: 複数セッションにわたるメモリ操作の高レベルマネージャー
- **MemorySession**: すべてのメソッドに actor_id/session_id を渡す必要がなくなるセッション固有のインターフェース
- **型安全性**: 作成時にセッションが特定の actor/session にバインドされる
- **カプセル化の向上**: セッション固有の操作がセッションオブジェクト内に含まれる
- **メモリフック**: セッションベースのアーキテクチャでエージェントフックが動作
- **会話の継続性**: MemoryManager と MemorySession による短期メモリ機能の維持

### MemorySession の主な利点：
1. **簡素化された API**: すべてのメソッド呼び出しで actor_id/session_id を渡す必要なし
2. **事前設定済みコンテキスト**: 作成時にセッションが特定の actor/session にバインド
3. **一貫したインターフェース**: すべてのセッション操作が同じ事前設定コンテキストを使用


## クリーンアップ（オプション）

In [ ]:
# MemoryManager を使用してメモリリソースを削除するにはコメントを解除
# try:
#     memory_manager.delete_memory(memory_id)
#     logger.info(f"✅ Deleted memory: {memory_id}")
# except Exception as e:
#     logger.error(f"Failed to delete memory: {e}")